In [1]:
# Dependencies
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
import os
import re

In [8]:
game_info_df=pd.read_csv("../data/boardgames_07022021.csv")
game_info_df

,objectid,name,yearpublished,sortindex,minplayers,maxplayers,minplaytime,maxplaytime,minage,min_community,...,boardgameartist,boardgamepublisher,boardgamehonor,boardgamecategory,boardgameversion,boardgamemechanic,boardgameexpansion,boardgamefamily,description,gamelink
0,174430,Gloomhaven,2017,1,1,4,60,120,14,3.0,...,"['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Albi Polska', '...",['2017 Best Science Fiction or Fantasy Board G...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Chinese edition', 'Czech edition', 'English ...","['Action Queue', 'Action Retrieval', 'Campaign...",['Gloomhaven: Assault on the Morning Star (Pro...,"['Category: Dungeon Crawler', 'Components: Min...",Vanquish monsters with strategic cardplay. Ful...,/boardgame/174430/gloomhaven
1,161936,Pandemic Legacy Season 1,2015,2,2,4,60,60,13,4.0,...,['Chris Quilliams'],"['Z-Man Games', 'Asterion Press', 'Devir', 'Fi...",['2015 Board Game Quest Awards Best Coop Game ...,"['Environmental', 'Medical']","['Chinese blue edition', 'Chinese red edition'...","['Action Points', 'Cooperative Game', 'Hand Ma...",['None'],"['Components: Map (Global Scale)', 'Game: Pand...",Mutating diseases are spreading around the wor...,/boardgame/161936/pandemic-legacy-season-1
2,224517,Brass Birmingham,2018,3,2,4,60,120,14,3.0,...,"['Lina Cossette', 'David Forest', 'Damien Mamm...","['Roxley', 'Arclight', 'BoardM Factory', 'Conc...",['2018 Board Game Quest Awards Best Production...,"['Economic', 'Industry / Manufacturing', 'Tran...","['Chinese edition', 'Czech/English edition', '...","['Hand Management', 'Income', 'Loans', 'Market...",['None'],"['Cities: Birmingham (England)', 'Country: Eng...","Build networks, grow industries, and navigate ...",/boardgame/224517/brass-birmingham
3,167791,Terraforming Mars,2016,4,1,5,120,120,12,3.0,...,['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Bru00e4dspel.se', '...",['2016 Board Game Quest Awards Best Strategy/E...,"['Economic', 'Environmental', 'Industry / Manu...","['Bulgarian edition', 'Chinese edition', 'Chin...","['Drafting', 'End Game Bonuses', 'Hand Managem...",['French Championship Promo Cards (fan expansi...,"['Components: Map (Global Scale)', 'Components...",Compete with rival CEOs to make Mars habitable...,/boardgame/167791/terraforming-mars
4,233078,Twilight Imperium Fourth Edition,2017,5,3,6,240,480,14,6.0,...,['Scott Schomburg'],"['Fantasy Flight Games', 'ADC Blackfire Entert...",['2017 Golden Geek Best Strategy Board Game No...,"['Civilization', 'Economic', 'Exploration', 'N...","['Chinese edition', 'Czech edition', 'English ...","['Action Drafting', 'Area Majority / Influence...",['Twilight Imperium: Fourth Edition u2013 Prop...,"['Components: Hexagonal Tiles', 'Components: M...","Build an intergalactic empire through trade, r...",/boardgame/233078/twilight-imperium-fourth-edi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,16518,Orbit,1959,29996,2,4,30,30,0,NaN,...,['(Uncredited)'],['Parker Brothers'],['None'],['Space Exploration'],"['First edition', 'Second edition']",['Roll / Spin and Move'],['None'],"['First edition', 'Second edition']",The object of this track game is to be the fir...,/boardgame/16518/orbit
29996,16519,Energie 21,2004,29997,2,4,0,0,0,NaN,...,"['Till Meyer', 'Nicole Stiehl']",['Spieltrieb'],['2005 Deutscher Lernspielpreis 9 years and up...,"['Educational', 'Environmental']",['German edition'],"['Cooperative Game', 'Trading']",['None'],['German edition'],The goal of Energie 21 - Mit Energie in die Zu...,/boardgame/16519/energie-21
29997,16520,Dragon's Lair,1983,29998,2,4,0,0,6,NaN,...,['(Uncredited)'],['Milton Bradley'],['None'],"['Adventure', 'Fantasy', 'Video Game Theme']","['English edition', 'English/French edition']","['Card Drafting', 'Roll / Spin and Move']",['None'],"['Creatures: Dragons', 'Theme: Arcade Video Ga...",Based on the ground-breaking (at the time) Las...,/boardgame/16520/dragons-lair
29998,16521,MAS

In [3]:
game_info_df.columns

Index(['objectid', 'name', 'yearpublished', 'sortindex', 'minplayers',
       'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'min_community',
       'max_community', 'totalvotes', 'playerage', 'languagedependence',
       'usersrated', 'average', 'baverage', 'stddev', 'avgweight',
       'numweights', 'numgeeklists', 'numtrading', 'numwanting', 'numcomments',
       'siteviews', 'numplays', 'numplays_month', 'news', 'blogs', 'weblink',
       'podcast', 'label', 'boardgamedesigner_cnt', 'boardgameartist_cnt',
       'boardgamepublisher_cnt', 'boardgamehonor_cnt', 'boardgamecategory_cnt',
       'boardgamemechanic_cnt', 'boardgameexpansion_cnt',
       'boardgameversion_cnt', 'boardgamefamily_cnt', 'boardgamedesigner',
       'boardgameartist', 'boardgamepublisher', 'boardgamehonor',
       'boardgamecategory', 'boardgameversion', 'boardgamemechanic',
       'boardgameexpansion', 'boardgamefamily', 'description', 'gamelink'],
      dtype='object')

In [9]:
game_info_df.drop_duplicates(subset=['objectid'], inplace=True)
game_info_df=game_info_df[['name', 'playerage', 'yearpublished',  'minplayers',
       'maxplayers', 'minplaytime', 'maxplaytime', 'minage', 'totalvotes', 'languagedependence',
       'usersrated', 'average', 'baverage', 'stddev', 'avgweight',
       'numweights', 'numgeeklists', 'numtrading', 'numwanting', 'numcomments',
       'siteviews', 'numplays', 'numplays_month', 'news', 'blogs', 'weblink',
       'podcast',  'boardgamedesigner_cnt', 'boardgameartist_cnt',
       'boardgamepublisher_cnt', 'boardgamehonor_cnt', 'boardgamecategory_cnt',
       'boardgamemechanic_cnt', 'boardgameexpansion_cnt',
       'boardgameversion_cnt', 'boardgamefamily_cnt', 'boardgamedesigner',
       'boardgameartist', 'boardgamepublisher', 'boardgamehonor',
       'boardgamecategory', 'boardgameversion', 'boardgamemechanic',
       'boardgameexpansion', 'boardgamefamily', 'description']].copy()
game_info_df.dropna(inplace=True)
game_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29936 entries, 0 to 29999
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    29936 non-null  object 
 1   playerage               29936 non-null  object 
 2   label                   29936 non-null  object 
 3   yearpublished           29936 non-null  int64  
 4   minplayers              29936 non-null  int64  
 5   maxplayers              29936 non-null  int64  
 6   minplaytime             29936 non-null  int64  
 7   maxplaytime             29936 non-null  int64  
 8   minage                  29936 non-null  int64  
 9   totalvotes              29936 non-null  int64  
 10  languagedependence      29936 non-null  int64  
 11  usersrated              29936 non-null  int64  
 12  average                 29936 non-null  float64
 13  baverage                29936 non-null  float64
 14  stddev                  29936 non-null

In [5]:
# convert unicode to printable format
a=game_info_df['name']
kk=[]
for b in a:
    b=re.sub('\s\s+', ' ', b)
    x=b.replace('\\u','/u').replace('\\', "").replace('/u','\\u')
    c=(x.encode('utf-8').decode('unicode-escape'))
    kk.append(c)
game_info_df['game_name']=kk

In [6]:
# convert description to JSON parsible format
a=game_info_df['description']
kk=[]
for b in a:
    b=re.sub(r'[^\x20-\x7F]',r'', b)
    b=b.replace('"',"|")
    kk.append(b)
game_info_df['game_description']=kk
game_info_df = game_info_df.replace(to_replace= r'\\', value= '', regex=True)

In [11]:
game_info_df.describe()

,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,totalvotes,languagedependence,usersrated,average,...,podcast,boardgamedesigner_cnt,boardgameartist_cnt,boardgamepublisher_cnt,boardgamehonor_cnt,boardgamecategory_cnt,boardgamemechanic_cnt,boardgameexpansion_cnt,boardgameversion_cnt,boardgamefamily_cnt
count,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000,...,29936.000000,29936.000000,29936.000000,29936.000000,29936.00000,29936.000000,29936.000000,29936.000000,29936.000000,29936.000000
mean,1955.310696,2.027258,5.827465,61.985068,80.555619,8.898049,11.069214,1.203501,605.154931,5.924815,...,1.457142,1.293994,1.106494,2.261725,0.39678,2.537847,2.485302,0.961885,2.941575,1.818546
std,302.712862,0.743408,66.344756,382.136750,452.870548,4.146720,54.999266,1.405164,3020.233612,1.578993,...,6.954266,0.762613,4.033087,4.593997,1.73652,1.369221,2.068297,6.238904,11.050243,2.058801
min,-3500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1990.000000,2.000000,3.000000,20.000000,20.000000,7.000000,0.000000,0.000000,19.000000,5.388105,...,0.000000,1.000000,0.000000,1.000000,0.00000,2.000000,1.000000,0.000000,1.000000,1.000000
50%,2004.000000,2.000000,4.000000,30.000000,45.000000,10.000000,2.000000,1.000000,63.000000,6.192615,...,0.000000,1.000000,1.000000,1.000000,0.00000,2.000000,2.000000,0.000000,1.000000,1.000000
75%,2014.000000,2.000000,6.000000,60.000000,90.000000,12.000000,5.000000,2.000000,224.000000,6.878975,...,1.000000,1.000000,1.000000,2.000000,0.00000,3.000000,3.000000,0.000000,3.000000,3.000000
max,2022.000000,10.000000,11299.000000,60000.000000,60000.000000,25.000000,2108.000000,5.000000,104891.000000,10.000000,...,588.000000,34.000000,509.000000,195.000000,53.00000,14.000000,20.000000,513.000000,1059.000000,104.000000
